In [ ]:
import matplotlib.pyplot as plt
import h5py
import rotation_mio as rot
import numpy as np
import time_conversion as time
import z50 as z50_prof

ID = np.loadtxt('../_data/my_halos.dat')[:,0]

edades = (4,7,10)
colors = ('b','k','r')

path = '/store/erebos/omarioni/_simulations/snap_127/'

for i in range(14):

    snap = h5py.File(path + 'subhalo_'+str('%d'%ID[i])+'.h5py', 'r')

    print('subhalo_'+str('%d'%ID[i]))

    aexp = snap['/Time'][()]
    h    = snap['/h'][()]
    Rvir = snap['/R200'][()]
    Om_L = snap['/Omega_Lambda'][()]
    Om_M = snap['/Omega_0'][()]

    pstr = snap['/Str/Coordinates'][()]
    mstr = snap['/Str/Masses'][()]
    vstr = snap['/Str/Velocities'][()]

    met  = snap['/Str/Total_Metallicity'][()]
    sft  = snap['/Str/FormationTime'][()]

    amask, = np.where(sft>0) #aca puede haber particulas con age<0, esas no son estrellas son gas cells.
    ages = time.conv2(sft[amask], h, Om_L, Om_M) #transformo el tiempo de formacion de factor de escala a Gyr
    age  = np.max(ages) - ages #lo resto asi saco la edad de las particulas (inverso al tiempo de form)

    Ztot = met[amask]/0.0127 #para pasarlo a metalicidad solar (ver Illustris data)

    xstr = pstr[amask,0]*aexp/h
    ystr = pstr[amask,1]*aexp/h
    zstr = pstr[amask,2]*aexp/h
    rstr = np.sqrt(xstr**2+ystr**2+zstr**2)

    v_x = vstr[amask,0] *np.sqrt(aexp)
    v_y = vstr[amask,1] *np.sqrt(aexp)
    v_z = vstr[amask,2] *np.sqrt(aexp)

    #----------------------masas----------------------------
    mstr = mstr[amask]*1e10/h

    rgal = 0.15*Rvir*aexp/h
    
    print('rgal = ', rgal)

    limit, = np.where(rstr < rgal)
    rsort = np.argsort(rstr[limit])
    Mc = np.cumsum(mstr[limit][rsort])
    Mgal = Mc[-1]
    limit50, = np.where(Mc < Mgal/2)
    r50 = rstr[limit][rsort][limit50][-1]

    veloc,=np.where(rstr < r50/2.)

    #----------componentes de la velocidad del centro de masa------------
    vxcm = sum(mstr[veloc]*v_x[veloc])/sum(mstr[veloc])
    vycm = sum(mstr[veloc]*v_y[veloc])/sum(mstr[veloc])
    vzcm = sum(mstr[veloc]*v_z[veloc])/sum(mstr[veloc])

    vx = v_x - vxcm
    vy = v_y - vycm
    vz = v_z - vzcm

    e1x,e2x,e3x,e1y,e2y,e3y,e1z,e2z,e3z = rot.rot1(mstr,xstr,ystr,zstr,vx,vy,vz,r50/2.)

    xn_str = e1x*xstr + e1y*ystr + e1z*zstr
    yn_str = e2x*xstr + e2y*ystr + e2z*zstr
    zn_str = e3x*xstr + e3y*ystr + e3z*zstr
    rn_str = np.sqrt(xn_str**2 + yn_str**2 + zn_str**2)

    Rstr = np.sqrt(xn_str**2 + yn_str**2)

    mask,  = np.where(rn_str <= 1.2*rgal)
    
    w0,  = np.where(Ztot[mask]>0) ##aca saco los FeH negativos para que no explote el log
    Z_total = np.log10(Ztot[mask][w0])
    
    R = Rstr[mask][w0]
    z = zn_str[mask][w0]
    m = mstr[mask][w0]
    edad = age[mask][w0]
    
    Id = ''
    for aux in list(str('%d'%ID[i]))[-2:]:
        Id += aux
        
    nbin = np.int_(rgal/0.8)
    print(nbin)
    # nbin=30
    
    fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(10, 8), sharex=True)
    fig.subplots_adjust(bottom=0.10, left =0.12, right = 0.96, top = 0.92,hspace=0.02)
    
    for k in range(3):

        age_bin,  = np.where((edad > edades[k]-0.5) & (edad < edades[k]+0.5))

        R_bin = R[age_bin]
        z_bin = z[age_bin]
        m_bin = m[age_bin]
        
        med, z50, nodos, p25, p75 = z50_prof.HMSH_log(R_bin,z_bin,m_bin,nbin)
        
        ax[0].plot(med,z50, ls='--',color=colors[k], label='$z_{50}$, Age='+str('%s'%edades[k])+'Gyr')
        ax[0].plot(med,-z50,ls='--',color=colors[k])
        
        ax[1].plot(med,z50, ls='--',color=colors[k], label='$z_{50}$, Age='+str('%s'%edades[k])+'Gyr')
        ax[1].plot(med,-z50,ls='--',color=colors[k])

    hb0 =ax[1].hexbin(R,z, Z_total,cmap='RdBu',vmin=-0.7,vmax=0.7)
    cbar0 = fig.colorbar(hb0,ax=ax[1])
    cbar0.set_label('log($Z_{tot}$/$Z_{\odot}$)',fontsize=22)
    cbar0.ax.tick_params(labelsize=22)
    cbar0.set_ticks([-0.6,-0.4,-0.2,0,0.2,0.4,0.6])

    ax[1].axhline(y=0,ls='--',color='darkgray')
    ax[1].set_ylim(-rgal/4,rgal/4)
    ax[1].set_xlim(0,rgal)
    ax[1].minorticks_on()
    ax[1].tick_params( labelsize=22)
    ax[1].tick_params('both', length=7, width=1.2,which='minor', direction='in', right='on',top='on')
    ax[1].tick_params('both', length=10, width=1.2,which='major', direction='in', right='on',top='on')
    ax[1].set_ylabel('z [kpc]',fontsize=22)
    ax[1].set_xlabel('R [kpc]',fontsize=22)


    hb1 =ax[0].hexbin(R,z,edad,cmap='RdBu_r',vmin=0,vmax=14)
    cbar1 = fig.colorbar(hb1,ax=ax[0])
    cbar1.set_label('Age [Gyr]',fontsize=22)
    cbar1.ax.tick_params( labelsize=22)
    cbar1.set_ticks([0,2,4,6,8,10,12])
    
    ax[0].axhline(y=0,ls='--',color='darkgray')
    ax[0].set_ylim(-rgal/4,rgal/4)
    ax[0].set_xlim(0,rgal)
    ax[0].minorticks_on()
    ax[0].tick_params( labelsize=22)
    ax[0].tick_params('both', length=7, width=1.2,which='minor', direction='in', right='on',top='on')
    ax[0].tick_params('both', length=10, width=1.2,which='major', direction='in', right='on',top='on')
    ax[0].set_ylabel('z [kpc]',fontsize=22)
    ax[0].legend(fontsize=15,loc=2,fancybox=True)
    ax[0].set_title('sh '+Id, fontsize =18)
    # ax[0].arrow(0.244,-3.5,0,-1.5,
    #     length_includes_head=True, head_width=rgal/60, head_length=0.5,
    #     lw=2,color='k')

    plt.show()

    # fig.savefig('../_imagenes/metallicity_ages/Ztot_ages_corect_s'+str('%d'%ID[i])+'.png',dpi=75, xxbox_inches='tight')